In [160]:
# TODO: Mejoras para este ejercicio:
# - Asegurar que los datos sean consistentes: si hay valores erróneos o nulos, establecer un random_state para garantizar reproducibilidad
# - Implementar pipelines para estructurar mejor el flujo de preprocesamiento y modelado, evitando la aplicación manual de cada paso
# - Optimizar el modelo ajustando hiperparámetros con técnicas como GridSearchCV o RandomizedSearchCV
# - Utilizar Regex para validaciones: códigos postales, teléfonos, emails, etc.
# - Crear variables derivadas como precio por metro cuadrado (precio_m2 = precio / superficie)
# - Geolocalización: obtener coordenadas con OpenStreetMap a partir de direcciones o códigos postales y utilizarlas para análisis espaciales
# - Visualizar las viviendas en un mapa interactivo con Folium o Plotly Express para identificar patrones geográficos en los precios
# - Clusterización de zonas con K-Means o DBSCAN para detectar patrones de precios por ubicación y segmentar mejor los inmuebles
# - Evitar data leakage: Dividir los datos en train/test antes de hacer encoding, eliminar outliers o escalar,
#   asegurando que las transformaciones se ajusten sólo con el conjunto de entrenamiento y luego se apliquen en test
# - Subir el proyecto final a Kaggle

## 1 - Carga de datos y revisión de la estructura del dataset

En este apartado se realiza una **primera exploración del dataset** para comprender su estructura, tipo de variables y posibles relaciones entre ellas. Este paso es fundamental para la correcta preparación de los datos antes del modelado.

### **1.1 - Importación de librerías y configuración**
Se importan las librerías esenciales para el análisis de datos, la visualización y el modelado, incluyendo **Pandas, NumPy, Matplotlib, Seaborn, SciPy y Scikit-Learn**. Además, se configuran algunos parámetros globales de visualización para mejorar la legibilidad de los gráficos.

### **1.2 - Carga del dataset**
Se carga el conjunto de datos en un **DataFrame de Pandas** desde un archivo CSV. Se incluye una referencia a la fuente del dataset.

### **1.3 - Examinar la estructura del dataset**
En esta fase se inspecciona la estructura general del dataset para entender su contenido y formato:

- **Visualización de las primeras y últimas filas** para detectar posibles errores en la carga de los datos.
- **Información general del dataset** (`df.info()`), que muestra el número de registros, tipos de datos y valores nulos.
- **Número total de filas y columnas** (`df.shape`).
- **Identificación de columnas numéricas y categóricas**, que ayudará en el preprocesamiento.
- **Recuento de valores únicos en variables categóricas**, útil para evaluar su diversidad.
- **Resumen estadístico de las variables numéricas**, para analizar su distribución, detectar valores atípicos y entender la escala de los datos.

### **1.4 - Comprobación de relaciones potenciales**
Para evaluar la viabilidad del modelado, se analizan las correlaciones entre variables numéricas:

- Se genera una **matriz de correlación** (`df.corr()`) para medir la relación entre variables.
- Se identifican **las variables con mayor impacto en `price`** basándose en la correlación:
  - `floor_built` (0.70)
  - `floor_area` (0.72)
  - `bedrooms` (0.51)
  - `bathrooms` (0.69)
- Se confirma la pertinencia de `balcony` como una **variable categórica de clasificación**, revisando la distribución de sus valores.


### 1.1 - Importación de librerías y otras configuraciones

In [161]:
# Cargar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
import folium
from geopy.distance import geodesic

# Configuraciones
%matplotlib inline
sns.set_theme(style="whitegrid", palette="viridis", font_scale=1.1)

### 1.2 - Carga del dataset

In [162]:
# Cargar el dataset local con Pandas
df = pd.read_csv("scripts/madrid_rent_with_coordinates.csv")

# Ref. https://www.kaggle.com/datasets/mapecode/madrid-province-rent-data

### 1.3 - Examinar la estructura del dataset

In [163]:
# Mostrar las primeras filas para una vista inicial del dataset
print("Primeras filas del dataset:")
display(df.head())

Primeras filas del dataset:


,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update,lat,lng
0,99440018,https://www.idealista.com/en/inmueble/99440018/,Studio flat for rent in luis cabrera,Studio,650,1.0,False,Madrid en Propiedad,30,NaN,...,False,False,False,"luis cabrera, Subdistrict Prosperidad, Distric...",Chamartín,Prosperidad,28002.0,7 November,40.444750,-3.671574
1,99440827,https://www.idealista.com/en/inmueble/99440827/,Flat / apartment for rent in calle de Pastora ...,Flat,1750,NaN,False,PUBLICASA MADRID,148,NaN,...,False,True,False,"Calle de Pastora Imperio, Subdistrict Castilla...",Chamartín,Castilla,28036.0,7 November,40.481725,-3.674384
2,97689853,https://www.idealista.com/en/inmueble/97689853/,Flat / apartment for rent in calle de Gabriel ...,Flat,1490,NaN,False,roomless,65,55.0,...,False,False,False,"Calle de Gabriel Lobo, 20, Subdistrict El Viso...",Chamartín,El Viso,28002.0,5 November,40.443449,-3.679917
3,97689852,https://www.idealista.com/en/inmueble/97689852/,Flat / apartment for rent in calle de Gabriel ...,Flat,900,NaN,False,roomless,50,40.0,...,False,False,False,"Calle de Gabriel Lobo, 20, Subdistrict El Viso...",Chamartín,El Viso,28002.0,5 November,40.443449,-3.679917
4,99399876,https://www.idealista.com/en/inmueble/99399876/,Flat / apartment for rent in El Viso,Flat,950,NaN,False,Spotahome,28,24.0,...,False,False,False,", Subdistrict El Viso, District Chamartín, Mad...",Chamartín,El Viso,NaN,6 November,40.449021,-3.686681


In [164]:
# Mostrar las últimas filas para identificar posibles problemas en la carga de datos
print("Últimas filas del dataset:")
display(df.tail())

Últimas filas del dataset:


,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update,lat,lng
9224,99283228,https://www.idealista.com/en/inmueble/99283228/,Flat / apartment for rent in calle Jerez de lo...,Flat,950,1.0,False,"EL PORTAL, GESTIÓN INMOBILIARIA",64,NaN,...,False,False,False,"Calle Jerez de los Caballeros, 5, Subdistrict ...",Barajas,Casco Histórico de Barajas,28042.0,23 October,40.465370,-3.595152
9225,99377540,https://www.idealista.com/en/inmueble/99377540/,Flat / apartment for rent in Alameda de Osuna,Flat,1800,1.0,False,Engel & Völkers Madrid,138,118.0,...,True,False,False,", Subdistrict Alameda de Osuna, District Baraj...",Barajas,Alameda de Osuna,28042.0,5 November,40.456178,-3.594880
9226,95831170,https://www.idealista.com/en/inmueble/95831170/,Flat / apartment for rent in Góndola,Flat,1350,1.0,False,Alquilar y Vender Madrid,114,104.0,...,False,True,False,"Góndola, Subdistrict Alameda de Osuna, Distric...",Barajas,Alameda de Osuna,28042.0,3 November,40.453432,-3.589196
9227,99405352,https://www.idealista.com/en/inmueble/99405352/,Flat / apartment for rent in calle Timón,Flat,850,NaN,False,Redpiso,64,51.0,...,False,False,False,"Calle Timón, Subdistrict Timón, District Baraj...",Barajas,Timón,28042.0,3 November,40.472818,-3.585470
9228,2139592,https://www.idealista.com/en/inmueble/2139592/,"Flat / apartment for rent in calle Bariloche, 1",Flat,950,1.0,True,NaN,70,NaN,...,True,True,False,"Calle Bariloche, 1, Urb. puerta coronales, Sub...",Barajas,Campo de las Naciones-Corralejos,28042.0,3 November,40.467742,-3.590706


In [165]:
# Obtener información general sobre el dataset, incluyendo tipos de datos y valores nulos
print("Información general del dataset:")
display(df.info())

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9229 entries, 0 to 9228
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   web_id             9229 non-null   int64  
 1   url                9229 non-null   object 
 2   title              9229 non-null   object 
 3   type               9229 non-null   object 
 4   price              9229 non-null   int64  
 5   deposit            5407 non-null   float64
 6   private_owner      9229 non-null   bool   
 7   professional_name  7622 non-null   object 
 8   floor_built        9229 non-null   int64  
 9   floor_area         3938 non-null   float64
 10  floor              8908 non-null   object 
 11  year_built         2893 non-null   float64
 12  orientation        4411 non-null   object 
 13  bedrooms           9229 non-null   int64  
 14  bathrooms          9229 non-null   int64  
 15  second_hand        9229 non-null   bool

None

In [166]:
# Mostrar el número de filas y columnas en el dataset
print("Número de filas y columnas en el dataset:")
print(df.shape)

Número de filas y columnas en el dataset:
(9229, 34)


In [167]:
# Identificar las columnas numéricas y categóricas
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [168]:
# Mostrar las columnas numéricas identificadas
print("Columnas numéricas:")
print(numerical_columns)

Columnas numéricas:
['web_id', 'price', 'deposit', 'floor_built', 'floor_area', 'year_built', 'bedrooms', 'bathrooms', 'postalcode', 'lat', 'lng']


In [169]:
# Mostrar las columnas categóricas identificadas
print("Columnas categóricas:")
print(categorical_columns)

Columnas categóricas:
['url', 'title', 'type', 'professional_name', 'floor', 'orientation', 'location', 'district', 'subdistrict', 'last_update']


In [170]:
# Contar valores únicos en variables categóricas
print("Valores únicos en variables categóricas:")
df[categorical_columns].nunique()

Valores únicos en variables categóricas:


url                  9229
title                5826
type                   10
professional_name    1580
floor                 198
orientation             4
location             5576
district              120
subdistrict           165
last_update           236
dtype: int64

In [171]:
# Describir estadísticamente las variables numéricas para analizar su distribución y posibles valores atípicos
print("Resumen estadístico de las variables numéricas:")
display(df.describe())

Resumen estadístico de las variables numéricas:


,web_id,price,deposit,floor_built,floor_area,year_built,bedrooms,bathrooms,postalcode,lat,lng
count,9.229000e+03,9229.000000,5407.000000,9229.000000,3938.000000,2893.000000,9229.000000,9229.000000,6834.000000,9229.000000,9229.000000
mean,9.133077e+07,1937.995883,1.470686,110.285405,94.357288,1975.491531,2.259508,1.787518,28076.947761,40.348891,-3.885809
std,2.016654e+07,1615.063308,0.622239,87.183901,72.822703,34.928909,1.332689,1.077126,171.686236,2.187182,4.094826
min,3.902730e+05,400.000000,1.000000,0.000000,0.000000,1800.000000,0.000000,1.000000,28001.000000,-34.466315,-122.282185
25%,9.578928e+07,1000.000000,1.000000,60.000000,54.000000,1960.000000,1.000000,1.000000,28009.000000,40.416723,-3.708353
50%,9.891848e+07,1400.000000,1.000000,85.000000,75.000000,1978.000000,2.000000,2.000000,28023.000000,40.432876,-3.693365
75%,9.929904e+07,2300.000000,2.000000,127.000000,110.000000,2003.000000,3.000000,2.000000,28043.000000,40.456408,-3.672290
max,9.944594e+07,25000.000000,6.000000,990.000000,995.000000,2022.000000,25.000000,20.000000,28950.000000,56.263920,9.501785


### 1.4 - Comprobar relaciones potenciales

In [172]:
# Se genera la matriz de correlación para analizar la relación entre las variables numéricas
print("Matriz de correlación entre variables numéricas:")
correlation_matrix = df[numerical_columns].corr()
display(correlation_matrix)

Matriz de correlación entre variables numéricas:


,web_id,price,deposit,floor_built,floor_area,year_built,bedrooms,bathrooms,postalcode,lat,lng
web_id,1.000000,0.011329,-0.048291,0.013207,0.047059,-0.033324,0.042937,0.016238,0.029412,-0.009297,-0.007663
price,0.011329,1.000000,0.059232,0.700145,0.728005,0.019199,0.515126,0.693651,-0.090495,-0.002525,-0.002353
deposit,-0.048291,0.059232,1.000000,0.109606,0.054622,0.030535,0.064656,0.114684,0.023844,-0.023048,-0.030092
floor_built,0.013207,0.700145,0.109606,1.000000,0.913232,0.104845,0.729301,0.803921,0.121317,-0.010606,-0.017718
floor_area,0.047059,0.728005,0.054622,0.913232,1.000000,0.070795,0.735387,0.772292,0.093523,0.015441,-0.003767
year_built,-0.033324,0.019199,0.030535,0.104845,0.070795,1.000000,0.047959,0.112018,0.219147,0.025375,-0.000908
bedrooms,0.042937,0.515126,0.064656,0.729301,0.735387,0.047959,1.000000,0.738524,0.118242,-0.001036,-0.001875
bathrooms,0.016238,0.693651,0.114684,0.803921,0.772292,0.112018,0.738524,1.000000,0.079569,-0.008114,-0.020107
postalcode,0.029412,-0.090495,0.023844,0.121317,0.093523,0.219147,0.118242,0.079569,1.000000,0.011623,0.003252
lat,-0.009297,-0.002525,-0.023048,-0.010606,0.015441,0.025375,-0.001036,-0.008114,0.011623,1.000000,0.699391


In [173]:
# Identificar relaciones relevantes para modelado

# Basándonos en la matriz de correlación, seleccionamos las variables con mayor impacto en 'price'
# Según el análisis previo, las variables con correlación más fuerte con 'price' son:
# - floor_built (0.70)
# - floor_area (0.72)
# - bedrooms (0.51)
# - bathrooms (0.69)
# Otras variables tienen correlaciones insignificantes (<0.01) y no se consideran para modelado

key_relationships = ['price', 'floor_built', 'floor_area', 'bedrooms', 'bathrooms']
print("Relaciones clave para la predicción de price:")
display(df[key_relationships].corr())

Relaciones clave para la predicción de price:


,price,floor_built,floor_area,bedrooms,bathrooms
price,1.000000,0.700145,0.728005,0.515126,0.693651
floor_built,0.700145,1.000000,0.913232,0.729301,0.803921
floor_area,0.728005,0.913232,1.000000,0.735387,0.772292
bedrooms,0.515126,0.729301,0.735387,1.000000,0.738524
bathrooms,0.693651,0.803921,0.772292,0.738524,1.000000


In [174]:
# Se selecciona balcony como variable de clasificación
# Se analiza la frecuencia de los valores presentes en balcony para entender su distribución
print("Frecuencia de valores en la variable balcony:")
df['balcony'].value_counts()

Frecuencia de valores en la variable balcony:


balcony
False    7579
True     1650
Name: count, dtype: int64

## 2 - Limpieza y validación de los datos

### 2.1 - Identificación y manejo de valores nulos


In [175]:
# Contamos los valores nulos en cada columna
print("Valores nulos en el dataset:")
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Valores nulos en el dataset:


deposit              3822
professional_name    1607
floor_area           5291
floor                 321
year_built           6336
orientation          4818
district              133
subdistrict           771
postalcode           2395
dtype: int64

In [176]:
# # Eliminar columnas con más del 50% de valores nulos
# threshold_col = 0.5
# null_percent = df.isnull().mean()
# cols_to_drop = null_percent[null_percent > threshold_col].index
#
# df.drop(columns=cols_to_drop, inplace=True)
# print(f"Columnas eliminadas por tener más del {threshold_col * 100}% de valores nulos:")
# print(list(cols_to_drop))

Columnas eliminadas por tener más del 50.0% de valores nulos:
['floor_area', 'year_built', 'orientation']


In [177]:
# # Eliminar filas con más del 50% de valores nulos
# threshold_row = 0.5  # 50% de valores nulos
# row_null_percent = df.isnull().mean(axis=1)
# rows_to_drop = row_null_percent[row_null_percent > threshold_row].index
#
# df.drop(index=rows_to_drop, inplace=True)
# print(f"Filas eliminadas por tener más del {threshold_row * 100}% de valores nulos: {len(rows_to_drop)}")

Filas eliminadas por tener más del 50.0% de valores nulos: 0


In [178]:
# # Conteo de valores nulos por columna después de la limpieza
# print("Valores nulos después de la limpieza:")
# df.isnull().sum()

Valores nulos después de la limpieza:


web_id                  0
url                     0
title                   0
type                    0
price                   0
deposit              3822
private_owner           0
professional_name    1607
floor_built             0
floor                 321
bedrooms                0
bathrooms               0
second_hand             0
lift                    0
garage_included         0
furnished               0
equipped_kitchen        0
fitted_wardrobes        0
air_conditioning        0
terrace                 0
balcony                 0
storeroom               0
swimming_pool           0
garden_area             0
location                0
district              133
subdistrict           771
postalcode           2395
last_update             0
lat                     0
lng                     0
dtype: int64